In [60]:
import pandas as pd 
import numpy as np
import statsmodels.api as sm

from dax.help_functions.get_dax_data import get_prepared_data
from evaluation.help_functions.prepare_data import most_recent_thursday, split_time
from dax.help_functions.calculate_returns import calculate_returns
from evaluation.help_functions.prepare_data import next_working_days

daxdata = get_prepared_data()
daxdata 

,Close,LogRetLag1,LogRetLag2,LogRetLag3,LogRetLag4,LogRetLag5
Date,,,,,,
2019-09-10 00:00:00+02:00,12268.709961,0.347914,0.629422,1.163586,2.006097,2.960150
2019-09-11 00:00:00+02:00,12359.070312,0.733812,1.081725,1.363234,1.897398,2.739909
2019-09-12 00:00:00+02:00,12410.250000,0.413251,1.147063,1.494977,1.776485,2.310649
2019-09-13 00:00:00+02:00,12468.530273,0.468515,0.881766,1.615578,1.963491,2.245000
2019-09-16 00:00:00+02:00,12380.309570,-0.710062,-0.241547,0.171704,0.905516,1.253429
...,...,...,...,...,...,...
2023-11-16 00:00:00+01:00,15786.610352,0.243797,1.096667,2.837245,3.560679,2.788120
2023-11-17 00:00:00+01:00,15919.160156,0.836129,1.079926,1.932796,3.673374,4.396808
2023-11-20 00:00:00+01:00,15901.330078,-0.112067,0.724062,0.967860,1.820729,3.561307


Quantile Regression for t+1

In [61]:
X = pd.DataFrame(daxdata.iloc[:-1]['LogRetLag1'].copy().abs())
X.insert(0,column='intercept',value=np.ones(shape= (len(daxdata)-1)))

Y = daxdata[['LogRetLag1']].shift(-1).iloc[:-1].copy()
Y.rename(columns={'LogRetLag1':"lr1dayahead"})

,lr1dayahead
Date,
2019-09-10 00:00:00+02:00,0.733812
2019-09-11 00:00:00+02:00,0.413251
2019-09-12 00:00:00+02:00,0.468515
2019-09-13 00:00:00+02:00,-0.710062
2019-09-16 00:00:00+02:00,-0.062209
...,...
2023-11-15 00:00:00+01:00,0.243797
2023-11-16 00:00:00+01:00,0.836129
2023-11-17 00:00:00+01:00,-0.112067


In [86]:
# model
quantiles = [0.025, 0.25, 0.5, 0.75, 0.975]
model_qr = sm.QuantReg(endog=Y,exog=X)
R_t = pd.DataFrame(daxdata.iloc[-1:]['LogRetLag1'].copy().abs())
R_t.insert(0, column='intercept', value=1)

#create frame 
column_names = [f'q{q}' for q in [0.025, 0.25, 0.5, 0.75, 0.975]]
dates = next_working_days(max(daxdata.index), 5)

quantiles_to_add = pd.DataFrame(index=dates, columns=column_names)

for q in quantiles:
    model_temp = model_qr.fit(q=q)
    # Calculate forecasts for R_t using the fitted model for the current quantile
    forecast_temp = model_temp.predict(R_t)
    quantiles_to_add.loc[dates[0]][f'q{q}'] = forecast_temp[0]

quantiles_to_add

,q0.025,q0.25,q0.5,q0.75,q0.975
2023-11-23,-2.678742,-0.474546,0.044688,0.631166,2.278425
2023-11-24,NaN,NaN,NaN,NaN,NaN
2023-11-27,NaN,NaN,NaN,NaN,NaN
2023-11-28,NaN,NaN,NaN,NaN,NaN
2023-11-29,NaN,NaN,NaN,NaN,NaN


In [ ]:
quantile_df = pd.DataFrame(index=dates, columns=column_names)
quantile_df.index.name = 'date_time'

date_st = daxdata.index[-1].strftime('%Y-%m-%d')
quantile_df.insert(0, 'forecast_date', date_st)
quantile_df.insert(1, 'target', 'DAX')
quantile_df.insert(2, "horizon", [str(i) + " day" for i in (1, 2, 5, 6, 7)])

Do it for all forecast horizons

In [95]:
# create forecast frame
column_names = [f'q{q}' for q in [0.025, 0.25, 0.5, 0.75, 0.975]]
dates = next_working_days(max(daxdata.index), 5)
quantile_df = pd.DataFrame(index=dates, columns=column_names)

R_t = pd.DataFrame(daxdata.iloc[-1:]['LogRetLag1'].copy().abs())
R_t.insert(0, column='intercept', value=1)

quantiles = [0.025, 0.25, 0.5, 0.75, 0.975]

for h in range(1,6): 
    print(f'{h}th Iteration')
    X = pd.DataFrame(daxdata.iloc[:-h][f'LogRetLag{h}'].copy().abs())
    X.insert(0, column='intercept', value=np.ones(shape=(len(daxdata)-h)))

    Y = daxdata[[f'LogRetLag{h}']].shift(-h).iloc[:-h].copy()
    Y.rename(columns={'LogRetLag1': "lr1dayahead"})

    model_qr_temp = sm.QuantReg(endog=Y, exog=X)

    for q in quantiles:
        model_quantile = model_qr_temp.fit(q=q)
        # Calculate forecasts for R_t using the fitted model for the current quantile
        forecast_temp = model_quantile.predict(R_t)
        quantile_df.loc[dates[h-1]][f'q{q}'] = forecast_temp[0]

quantile_df.insert(0, 'forecast_date', date_st)
quantile_df.insert(1, 'target', 'DAX')
quantile_df.insert(2, "horizon", [str(i) + " day" for i in (1, 2, 5, 6, 7)])
quantile_df.index.name = "date_time"
quantile_df

1th Iteration
2th Iteration
3th Iteration
4th Iteration
5th Iteration


,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
date_time,,,,,,,,
2023-11-23,2023-11-22,DAX,1 day,-2.678742,-0.474546,0.044688,0.631166,2.278425
2023-11-24,2023-11-22,DAX,2 day,-3.012408,-0.60897,0.1178,0.774698,2.887623
2023-11-27,2023-11-22,DAX,5 day,-3.967045,-0.679798,0.245021,1.067693,3.715867
2023-11-28,2023-11-22,DAX,6 day,-4.445875,-0.771721,0.210363,1.282541,3.462121
2023-11-29,2023-11-22,DAX,7 day,-4.722503,-0.774133,0.323645,1.233462,3.661439
